In [ ]:
#!pip install bs4 --user

In [180]:
# Import dependencies
import pandas as pd
import pymongo
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time

In [181]:
# Configure ChromeDriver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\Jeawills\.wdm\drivers\chromedriver\win32\87.0.4280.88]


# NASA Mars News

In [187]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')



In [188]:
article_container = news_soup.find('ul', class_='item_list')
article_container
    


<ul class="item_list"><li class="slide"><div class="image_and_description_container"><a href="/news/8822/a-martian-roundtrip-nasas-perseverance-rover-sample-tubes/" target="_self"><div class="rollover_description"><div class="rollover_description_inner">Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. </div><div class="overlay_arrow"><img alt="More" src="/assets/overlay-arrow.png"/></div></div><div class="list_image"><img alt="A tray holding 39 sample tubes – each protected in a gold-colored sheath – is installed in NASA's Perseverance rover in this picture taken at the agency's Kennedy Space Center on May 21, 2020." src="/system/news_items/list_view_images/8822_1-PIA24304-CatScanMars-320x240.gif"/></div><div class="bottom_gradient"><div><h3>A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes</h3></div></div></a><div class="list_text"><div class="list_date">Decembe

In [189]:
headline_date = article_container.find('div', class_='list_date').text
headline_date

'December 22, 2020'

In [190]:
news_title = article_container.find('div', class_='content_title').find('a').text
news_title

"A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes"

In [191]:
news_p = article_container.find('div', class_='article_teaser_body').text
news_p
    


"Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. "

In [192]:
def mars_news():
    url = 'https://mars.nasa.gov/news/'
    browser.visit(url)
    
    time.sleep(3) # Sleep for 3 seconds

    html = browser.html
    news_soup = BeautifulSoup(html, 'html.parser')

    article_container = news_soup.find('ul', class_='item_list')

    headline_date = article_container.find('div', class_='list_date').text
    headline_date
    
    news_title = article_container.find('div', class_='content_title').find('a').text
    news_p = article_container.find('div', class_='article_teaser_body').text

    return news_title, news_p

# JPL Mars Space Images - Featured Image

In [222]:
def featured_image():
    base_url = 'https://www.jpl.nasa.gov'

    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)

    html = browser.html
    img_soup = BeautifulSoup(html, 'html.parser')

    
    # Method of clicking the FULL TEXT button and pulling the image
    try:
        full_image_elem = browser.find_by_id('full_image')[0]
        full_image_elem.click()

        html = browser.html
        img_soup = BeautifulSoup(html, 'html.parser')

        img_rel_url = img_soup.find('img', class_='fancybox-image')['src']
        
    except Exception as e:
        print(e)

    featured_image_url  = f'{base_url}{img_rel_url}'
      
    return  featured_image_url

In [223]:
test = featured_image()
test

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20316_ip.jpg'

# Mars Facts

In [224]:
def mars_facts():
    url = 'https://space-facts.com/mars/'
    browser.visit(url)

    mars_facts_df = pd.read_html(url)
    mars_facts_df = mars_facts_df[0]
    mars_facts_df.columns = ['Description', 'Mars']
    mars_facts_df

    mars_facts_html = mars_facts_df.to_html(classes='table table-striped')
    
    return mars_facts_html

# Mars Hemispheres

In [225]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Retrieve page with the requests module 

browser.visit(url) 

html = browser.html  

#Use Beautiful soup to find Hemisphere titles in text

elements_soup = BeautifulSoup(html, 'html.parser')
elements_div = elements_soup.find("div", class_ = "collapsible")
elements = elements_div.find_all("h3")
elements = [element.text for element in elements]
elements




['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [226]:
# Use Splinter to click each title and scrap for the full hemisphere image  

hemisphere_image_urls = [] 
# empty list to be appended by dictionaries  

for x in range(len(elements)): 

    #parse for each title element          

    browser.links.find_by_partial_text(f'{elements[x]}').click()          

    title = elements[x]     

    image_url = browser.find_by_css('a').links.find_by_partial_text("Sample")['href']          

    hemisphere_image_urls.append(                                 
    {"title": f'{title}', "img_url": f'{image_url}'}
                                )
#return to home page
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    html = browser.html


In [230]:
def mars_hemispheres():    
    # Retrieve page with the requests module 
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

    browser.visit(url) 

    html = browser.html  

    #Use Beautiful soup to find Hemisphere titles in text

    elements_soup = BeautifulSoup(html, 'html.parser')
    elements_div = elements_soup.find("div", class_ = "collapsible")
    elements = elements_div.find_all("h3")
    elements = [element.text for element in elements]
    elements

    # Use Splinter to click each title and scrap for the full hemisphere image  

    hemisphere_image_urls = [] 

    # empty list to be appended by dictionaries  

    for x in range(len(elements)): 

        #parse for each title element          

        browser.links.find_by_partial_text(f'{elements[x]}').click()          
       
        title = elements[x]     

        image_url = browser.find_by_css('a').links.find_by_partial_text("Sample")['href']          

        hemisphere_image_urls.append(                                 
        {"title": f'{title}', "img_url": f'{image_url}'}
                                    )
        #return to home page
        url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
        browser.visit(url)
       
        html = browser.html
        
    return hemisphere_image_urls

In [231]:
urls = mars_hemispheres()
urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

# Insert into Mongo DB

In [232]:
def scrape_all():

    # Populate variables from the functions
    news_title, news_p = mars_news()
    featured_img_url = featured_image()
    mars_facts_html = mars_facts()
    hemispheres_mars= mars_hemispheres()

    # Assemble the document to insert into the database
    nasa_document = {
        'news_title': news_title,
        'news_paragraph': news_p,
        'featured_url': featured_img_url,
        'mars_facts': mars_facts_html,
        'hemispheres': hemispheres_mars
    }

    return nasa_document

In [233]:
doc = scrape_all()
doc

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_paragraph': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition. ",
 'featured_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20316_ip.jpg',
 'mars_facts': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Mars</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      

In [234]:
# Connect to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to mars_app database
db = client.mars_app

# Connect to mars collection
mars = db.mars

# Gather document to insert
data_document = scrape_all()
# Insert into the mars collection
# mars.insert_one(data_document)

# Upsert into the mars collection (prefed to avoid duplicates)
mars.update_one({}, {'$set': data_document}, upsert=True)